In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mltiverse-join-comp/MLtiVerse/sample_submission.csv
/kaggle/input/mltiverse-join-comp/MLtiVerse/train.csv
/kaggle/input/mltiverse-join-comp/MLtiVerse/test.csv


importing the data needed for hackathon

In [2]:
train_data=pd.read_csv('/kaggle/input/mltiverse-join-comp/MLtiVerse/train.csv')
test_data=pd.read_csv('/kaggle/input/mltiverse-join-comp/MLtiVerse/test.csv')
sub_data=pd.read_csv('/kaggle/input/mltiverse-join-comp/MLtiVerse/sample_submission.csv')

/tmp/ipykernel_21/1580012756.py:2: DtypeWarning: Columns (61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  test_data=pd.read_csv('/kaggle/input/mltiverse-join-comp/MLtiVerse/test.csv')


viewing the id column

In [3]:
train_data['Photo'].head()

0    https://cdn.sofifa.com/players/195/427/17_60.png
1    https://cdn.sofifa.com/players/203/539/17_60.png
2    https://cdn.sofifa.com/players/232/898/20_60.png
3    https://cdn.sofifa.com/players/215/502/17_60.png
4    https://cdn.sofifa.com/players/222/263/18_60.png
Name: Photo, dtype: object

going to start with a very basic xgboost submission selecting what i think are some important columns

changing the indexes of train and test

In [4]:
# Set the index column as 'Photo' in train_data
train_data.set_index('Photo', inplace=True)

# Set the index column as 'Photo' in test_data
test_data.set_index('Photo', inplace=True)

importing xgboost model

In [5]:
from xgboost import XGBRegressor
model=XGBRegressor()

checking which columns are available for modelling

In [6]:
train_data.columns

Index(['Unnamed: 0', 'ID', 'Name', 'Age', 'Nationality', 'Flag', 'Overall',
       'Potential', 'Club', 'Club Logo', 'Value', 'Wage', 'Special',
       'Preferred Foot', 'International Reputation', 'Weak Foot',
       'Skill Moves', 'Work Rate', 'Body Type', 'Real Face', 'Position',
       'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until',
       'Height', 'Weight', 'Crossing', 'Finishing', 'HeadingAccuracy',
       'ShortPassing', 'Volleys', 'Dribbling', 'Curve', 'FKAccuracy',
       'LongPassing', 'BallControl', 'Acceleration', 'SprintSpeed', 'Agility',
       'Reactions', 'Balance', 'ShotPower', 'Jumping', 'Stamina', 'Strength',
       'LongShots', 'Aggression', 'Interceptions', 'Positioning', 'Vision',
       'Penalties', 'Composure', 'Marking', 'StandingTackle', 'SlidingTackle',
       'GKDiving', 'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes',
       'Best Position', 'Best Overall Rating', 'year', 'Release Clause',
       'DefensiveAwareness'],
      dtyp

In [7]:
train_data['Release Clause'].isna().sum()

25095

In [8]:
train_data['Potential'].head()

Photo
https://cdn.sofifa.com/players/195/427/17_60.png    68
https://cdn.sofifa.com/players/203/539/17_60.png    58
https://cdn.sofifa.com/players/232/898/20_60.png    76
https://cdn.sofifa.com/players/215/502/17_60.png    86
https://cdn.sofifa.com/players/222/263/18_60.png    75
Name: Potential, dtype: int64

In [9]:
train_data.columns

Index(['Unnamed: 0', 'ID', 'Name', 'Age', 'Nationality', 'Flag', 'Overall',
       'Potential', 'Club', 'Club Logo', 'Value', 'Wage', 'Special',
       'Preferred Foot', 'International Reputation', 'Weak Foot',
       'Skill Moves', 'Work Rate', 'Body Type', 'Real Face', 'Position',
       'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until',
       'Height', 'Weight', 'Crossing', 'Finishing', 'HeadingAccuracy',
       'ShortPassing', 'Volleys', 'Dribbling', 'Curve', 'FKAccuracy',
       'LongPassing', 'BallControl', 'Acceleration', 'SprintSpeed', 'Agility',
       'Reactions', 'Balance', 'ShotPower', 'Jumping', 'Stamina', 'Strength',
       'LongShots', 'Aggression', 'Interceptions', 'Positioning', 'Vision',
       'Penalties', 'Composure', 'Marking', 'StandingTackle', 'SlidingTackle',
       'GKDiving', 'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes',
       'Best Position', 'Best Overall Rating', 'year', 'Release Clause',
       'DefensiveAwareness'],
      dtyp

manually selecting some important features in the dataset for modelling

In [10]:
train_1=train_data[['Potential','Overall','Wage','International Reputation','Age']]
test_1=test_data[['Potential','Overall','Wage','International Reputation','Age']]

making some changes to the wage column

In [11]:
train_1

,Potential,Overall,Wage,International Reputation,Age
Photo,,,,,
https://cdn.sofifa.com/players/195/427/17_60.png,68,61,€2K,1.0,22
https://cdn.sofifa.com/players/203/539/17_60.png,58,57,€500,1.0,25
https://cdn.sofifa.com/players/232/898/20_60.png,76,71,€7K,1.0,23
https://cdn.sofifa.com/players/215/502/17_60.png,86,71,€10K,1.0,18
https://cdn.sofifa.com/players/222/263/18_60.png,75,64,€2K,1.0,19
...,...,...,...,...,...
https://cdn.sofifa.com/players/188/278/17_60.png,73,68,€20K,1.0,25
https://cdn.sofifa.com/players/239/649/18_60.png,58,58,€1K,1.0,28
https://cdn.sofifa.com/players/258/958/20_60.png,69,69,€8K,1.0,27


checking which data types are object

In [12]:
y_train=train_data['Value']

converting the wage column to float data type

In [13]:
import re
# Define a function to preprocess the 'Wage' column
def preprocess_wage(value):
    value = re.sub(r'€', '', value)  # Remove euro symbol
    if 'K' in value:
        value = float(re.sub(r'[^0-9.]', '', value)) * 1000  # Remove 'K' and multiply by 1000
    elif 'M' in value:
        value = float(re.sub(r'[^0-9.]', '', value)) * 1000000  # Remove 'M' and multiply by 1000000
    else:
        value = float(value)
    return value

# Apply the preprocessing function to 'Wage' column in train_1 dataset
train_1['Wage'] = train_1['Wage'].apply(preprocess_wage)

# Convert the column to float data type
train_1['Wage'] = train_1['Wage'].astype(float)

# Apply the preprocessing function to 'Wage' column in train_1 dataset
test_1['Wage'] = test_1['Wage'].apply(preprocess_wage)

# Convert the column to float data type
test_1['Wage'] = test_1['Wage'].astype(float)

#Applying it to y_train too
y_train=y_train.apply(preprocess_wage)

#Convert the column to float data type
y_train=y_train.astype(float)

/tmp/ipykernel_21/3838849759.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_1['Wage'] = train_1['Wage'].apply(preprocess_wage)
/tmp/ipykernel_21/3838849759.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_1['Wage'] = train_1['Wage'].astype(float)
/tmp/ipykernel_21/3838849759.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydat

In [14]:
train_1.head()

,Potential,Overall,Wage,International Reputation,Age
Photo,,,,,
https://cdn.sofifa.com/players/195/427/17_60.png,68,61,2000.0,1.0,22
https://cdn.sofifa.com/players/203/539/17_60.png,58,57,500.0,1.0,25
https://cdn.sofifa.com/players/232/898/20_60.png,76,71,7000.0,1.0,23
https://cdn.sofifa.com/players/215/502/17_60.png,86,71,10000.0,1.0,18
https://cdn.sofifa.com/players/222/263/18_60.png,75,64,2000.0,1.0,19


In [15]:
y_train.head()

Photo
https://cdn.sofifa.com/players/195/427/17_60.png     325000.0
https://cdn.sofifa.com/players/203/539/17_60.png     140000.0
https://cdn.sofifa.com/players/232/898/20_60.png    3200000.0
https://cdn.sofifa.com/players/215/502/17_60.png    2900000.0
https://cdn.sofifa.com/players/222/263/18_60.png     575000.0
Name: Value, dtype: float64

using gridsearchCV and modelling now

In [16]:
# from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import make_scorer, mean_squared_error
# # Define a custom scoring function for RMSE
# scorer = make_scorer(lambda y_true, y_pred: mean_squared_error(y_true, y_pred, squared=False), greater_is_better=False)
# model=XGBRegressor()
# param_grid = {'learning_rate':[0.05, 0.75, 0.1], 'n_estimators':[200,400,600,800,1000]}
# grid_search = GridSearchCV(model, param_grid, cv=5, scoring=scorer)

applying and fitting GridSearchCV on the training data

In [17]:
model=XGBRegressor()
model.fit(train_1, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [18]:
test_1.head()

,Potential,Overall,Wage,International Reputation,Age
Photo,,,,,
https://cdn.sofifa.com/players/212/198/22_60.png,89,88,250000.0,3.0,26
https://cdn.sofifa.com/players/209/658/22_60.png,88,87,140000.0,4.0,26
https://cdn.sofifa.com/players/176/580/22_60.png,88,88,135000.0,5.0,34
https://cdn.sofifa.com/players/192/985/22_60.png,91,91,350000.0,4.0,30
https://cdn.sofifa.com/players/224/334/22_60.png,84,84,45000.0,2.0,29


In [19]:
test_1.shape

(34370, 5)

storing predictions

In [20]:
pred=model.predict(test_1)

In [21]:
pred.shape

(34370,)

In [22]:
output = pd.DataFrame({'Value': pred})  # Create the DataFrame with the 'Value' column
output = output.set_index(test_1.index)  # Set the index column to test_1.index

now storing the final submission

In [23]:
output.head()

,Value
Photo,
https://cdn.sofifa.com/players/212/198/22_60.png,67480096.0
https://cdn.sofifa.com/players/209/658/22_60.png,57726600.0
https://cdn.sofifa.com/players/176/580/22_60.png,24389748.0
https://cdn.sofifa.com/players/192/985/22_60.png,89608232.0
https://cdn.sofifa.com/players/224/334/22_60.png,30389520.0


converting to a csv now

In [24]:
output.to_csv('submissions.csv')
print("Submission saved successfully")

Submission saved successfully


done for now